In [13]:
import formulas
import storage
import pandas as pd
import os
import re



if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_path + os.path.sep + 'sort_orders.csv'
    docs_dir = storage.get_docs_path(website_creator_path)
    
    df_formulas_by_year =  get_formulas_by_year_df(formula_file_path)
    sort_orders = pd.read_csv(order_file_path)
    
    storage.delete_directory_if_it_exists(docs_dir)

    # Create formula by year summary
    formula_by_year_directories_df = (
        df_formulas_by_year[['State', 'Sub category 1', 'Sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    storage.create_sub_directories_from_df(
        docs_dir, formula_by_year_directories_df)
    regex_id_for_formula_dirs = '.*' + os.path.sep + 'Formulas*'
    storage.create_index_files(base_dir=docs_dir, 
                               folder_regex=regex_id_for_formula_dirs, 
                               book_collapse=True, df_sort_orders=sort_orders)
    # # # formulas.create_formula_files(docs_dir, df_formulas)


    # Create index files for State level menus
    regex_id_for_state_dirs =os.path.sep + '.{3}$'
    storage.create_index_files(base_dir=docs_dir, 
                               folder_regex=regex_id_for_state_dirs,
                               book_collapse=True, df_sort_orders=sort_orders)

    # df_calculus = formulas.df_calculus_summary(df_formulas_by_year)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formulas.formulas_on_formula_sheet(df_formulas_by_year))
    # display(styler_calculus)

In [5]:


def create_formula_files(docs_dir, df_formulas):
    """Creates formula files in markdown format.  Files are created per state /
    subject  / categorycombination and stored in folders under docs_path
    according to this same combination"""

    formula_combination = (df_formulas[['State', 'Subject code', 'Category']].
                           drop_duplicates())

    for index, row in formula_combination.iterrows():
        formula_set_df = (formulas_by_state_subject_category_df(
            df_formulas, row['State'], row['Subject code'], row['Category']))
        formula_set_styler = (df_to_formula_styled_table(
            df=formula_set_df, col_widths={'Formula_1':300, 'Formula_2':400},
            display_col_headers = False))
        output_string =  '#  \n<br>\n' + formula_set_styler.to_html()
        file_name = (docs_dir + os.path.sep +'formulas by year' + 
                     os.path.sep  + row['State'] + os.path.sep  + 
                     str(row['Subject code']) + os.path.sep  + 
                     row['Category']  + '.md')
        with open(file_name, "w") as text_file:
            text_file.write(output_string)


In [12]:
df_formulas_by_year= get_formula_by_year_df(formula_file_path)
df_formulas_by_year[['State', 'Sub category 1', 'Sub category 2', 'Subject code', 'Category']].drop_duplicates()

,State,Sub category 1,Sub category 2,Subject code,Category
0,NSW,Formulas,By Year,Year 9,Indices
9,NSW,Formulas,By Year,Year 11 Adv,Logarithms
14,NSW,Formulas,By Year,Year 11 Adv,Differentiation
21,NSW,Formulas,By Year,Year 12 Adv,Differentiation
22,NSW,Formulas,By Year,Year 12 Adv,Integration
42,NSW,Formulas,By Year,Year 12 Ext 1,Differentiation
43,NSW,Formulas,By Year,Year 12 Ext 1,Integration
55,NSW,Formulas,By Year,Year 12 Adv,Statistics
62,NSW,Formulas,By Year,Year 9,Surds
